In [1]:
import sys
import time
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import matplotlib
matplotlib.use('TKAgg')
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve
from sklearn import metrics, svm
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import TensorBoard

def dataProc(X1,y1):
    X1 = pd.DataFrame(X1)
    X1=X1.to_numpy()
    y1 = y1.to_numpy()
    
    
    min_max_scaler = preprocessing.MinMaxScaler()
    X1 = min_max_scaler.fit_transform(X1)
    
    X1, y1 = shuffle(X1, y1, random_state=0)
    return X1,y1

E1012 23:24:23.536626  6832 ioloop.py:763] Exception in callback functools.partial(<function Kernel.enter_eventloop.<locals>.advance_eventloop at 0x000001AA5B7FF048>)
Traceback (most recent call last):
  File "c:\users\jacob\appdata\local\programs\python\python37\lib\site-packages\tornado\ioloop.py", line 743, in _run_callback
    ret = callback()
  File "c:\users\jacob\appdata\local\programs\python\python37\lib\site-packages\ipykernel\kernelbase.py", line 306, in advance_eventloop
    eventloop(self)
  File "c:\users\jacob\appdata\local\programs\python\python37\lib\site-packages\ipykernel\eventloops.py", line 238, in loop_tk
    app.tk.createfilehandler(stream.getsockopt(zmq.FD), READABLE, notifier)
AttributeError: '_tkinter.tkapp' object has no attribute 'createfilehandler'


## Loading and Processing Data

In [2]:
sysName = "IEEE14"
testType = "ModelsFinal"
numEpochs = 100
checkpoint_path = testType+"/Saved Models/"+sysName+"_models/"

X1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_40k_lowSparsity"+".csv")
y1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_40k_lowSparsity"+".csv")
X2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_40k_midSparsity"+".csv")
y2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_40k_midSparsity"+".csv")
X3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_40k_highSparsity"+".csv")
y3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_40k_highSparsity"+".csv")

Xv1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_10k_lowSparsity"+".csv")
yv1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_10k_lowSparsity"+".csv")
Xv2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_10k_midSparsity"+".csv")
yv2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_10k_midSparsity"+".csv")
Xv3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_10k_highSparsity"+".csv")
yv3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_10k_highSparsity"+".csv")

Xt1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_4k_lowSparsity"+".csv")
yt1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_4k_lowSparsity"+".csv")
Xt2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_4k_midSparsity"+".csv")
yt2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_4k_midSparsity"+".csv")
Xt3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_4k_highSparsity"+".csv")
yt3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_4k_highSparsity"+".csv")

X1,y1 = dataProc(X1,y1)
Xv1,yv1 = dataProc(Xv1,yv1)
Xt1,yt1 = dataProc(Xt1,yt1)
X2,y2 = dataProc(X2,y2)
Xv2,yv2 = dataProc(Xv2,yv2)
Xt2,yt2 = dataProc(Xt2,yt2)

X3,y3 = dataProc(X3,y3)
Xv3,yv3 = dataProc(Xv3,yv3)
Xt3,yt3 = dataProc(Xt3,yt3)

numFeatures = len(X1[0])

X = np.concatenate((X1, X2, X3), axis=0)
y = np.concatenate((y1, y2, y3), axis=0)
Xv = np.concatenate((Xv1, Xv2, Xv3), axis=0)
yv = np.concatenate((yv1, yv2, yv3), axis=0)
Xt = np.concatenate((Xt1, Xt2, Xt3), axis=0)
yt = np.concatenate((yt1, yt2, yt3), axis=0)

X, y = shuffle(X, y, random_state=0)
Xv, yv = shuffle(Xv, yv, random_state=0)
Xt, yt = shuffle(Xt, yt, random_state=0)

In [3]:
savedModels_path = 'Saved Models'

resultsLow = []
resultsMid = []
resultsHigh = []

## Training

In [4]:
NUM_EPOCHS = 50

In [5]:
num_layers = [2,3,4,5,6]
layer_sizes = [16,32,64,128]

for layer in num_layers:
    for layer_size in layer_sizes:
        NAME = "{}-layers-{}-nodes".format(layer, layer_size)
        LOGNAME = "{}-layers-{}-nodes-{}".format(layer, layer_size, int(time.time()))
        tensorboard = TensorBoard(log_dir='logs\{}'.format(LOGNAME))
        print(NAME)
        
        model = keras.Sequential()
        model.add(tf.keras.Input(shape=(numFeatures,)))
        
        ##Adding Layers
        for i in range(layer):
            model.add(  tf.keras.layers.Dense(layer_size,activation='relu')  )
        
        ##Output Layer
        model.add( tf.keras.layers.Dense(1,activation='sigmoid') )
        
        ##Compiling model
        model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics = ['accuracy'])
        
        ##Training Model
        model.fit(X,y, batch_size = 32,
                  epochs = NUM_EPOCHS,
                  validation_data=(Xv,yv),
                  callbacks = [tensorboard] )
        
        ##Saving Model
        model.save('SavedModels\{}.h5'.format(NAME))
        
        ##Testing Model
        results1 = model.evaluate(Xt1, yt1)
        results2 = model.evaluate(Xt2, yt2)
        results3 = model.evaluate(Xt3, yt3)
        
        resultsLow.append(results1)
        resultsMid.append(results2)
        resultsHigh.append(results3)

2-layers-16-nodes
Train on 119997 samples, validate on 29997 samples
Epoch 1/50
119997/119997 [==============================] - 5s 40us/sample - loss: 0.6902 - accuracy: 0.5147 - val_loss: 0.6789 - val_accuracy: 0.5422
Epoch 2/50
119997/119997 [==============================] - 4s 34us/sample - loss: 0.6801 - accuracy: 0.5504 - val_loss: 0.6826 - val_accuracy: 0.5021
Epoch 3/50
119997/119997 [==============================] - 4s 35us/sample - loss: 0.6578 - accuracy: 0.5832 - val_loss: 0.7166 - val_accuracy: 0.5121
Epoch 4/50
119997/119997 [==============================] - 4s 34us/sample - loss: 0.6416 - accuracy: 0.5947 - val_loss: 0.7585 - val_accuracy: 0.5080
Epoch 5/50
119997/119997 [==============================] - 4s 31us/sample - loss: 0.6332 - accuracy: 0.5976 - val_loss: 0.7929 - val_accuracy: 0.5130
Epoch 6/50
119997/119997 [==============================] - 4s 33us/sample - loss: 0.6283 - accuracy: 0.5995 - val_loss: 0.8540 - val_accuracy: 0.5156
Epoch 7/50
119997/119997 

119997/119997 [==============================] - 5s 38us/sample - loss: 0.6849 - accuracy: 0.5270 - val_loss: 0.7281 - val_accuracy: 0.4905
Epoch 4/50
119997/119997 [==============================] - 4s 34us/sample - loss: 0.6792 - accuracy: 0.5406 - val_loss: 0.7670 - val_accuracy: 0.4869
Epoch 5/50
119997/119997 [==============================] - 4s 33us/sample - loss: 0.6580 - accuracy: 0.5834 - val_loss: 0.8395 - val_accuracy: 0.4902
Epoch 6/50
119997/119997 [==============================] - 4s 34us/sample - loss: 0.6270 - accuracy: 0.6268 - val_loss: 0.9477 - val_accuracy: 0.5344
Epoch 7/50
119997/119997 [==============================] - 4s 33us/sample - loss: 0.5898 - accuracy: 0.6734 - val_loss: 1.1157 - val_accuracy: 0.5330
Epoch 8/50
119997/119997 [==============================] - 4s 33us/sample - loss: 0.5606 - accuracy: 0.6869 - val_loss: 1.4135 - val_accuracy: 0.5336
Epoch 9/50
119997/119997 [==============================] - 4s 33us/sample - loss: 0.5462 - accuracy: 0.6

Epoch 5/50
119997/119997 [==============================] - 5s 41us/sample - loss: 0.6475 - accuracy: 0.6135 - val_loss: 0.9281 - val_accuracy: 0.5422
Epoch 6/50
119997/119997 [==============================] - 4s 37us/sample - loss: 0.6036 - accuracy: 0.6640 - val_loss: 1.1884 - val_accuracy: 0.5725
Epoch 7/50
119997/119997 [==============================] - 4s 34us/sample - loss: 0.5764 - accuracy: 0.6724 - val_loss: 1.2503 - val_accuracy: 0.6022
Epoch 8/50
119997/119997 [==============================] - 4s 34us/sample - loss: 0.5605 - accuracy: 0.6861 - val_loss: 1.4633 - val_accuracy: 0.5943
Epoch 9/50
119997/119997 [==============================] - 4s 33us/sample - loss: 0.5468 - accuracy: 0.7057 - val_loss: 1.8453 - val_accuracy: 0.5183
Epoch 10/50
119997/119997 [==============================] - 4s 34us/sample - loss: 0.5359 - accuracy: 0.7158 - val_loss: 1.6460 - val_accuracy: 0.6083
Epoch 11/50
119997/119997 [==============================] - 4s 34us/sample - loss: 0.5255 - 

119997/119997 [==============================] - 4s 35us/sample - loss: 0.5572 - accuracy: 0.6654 - val_loss: 2.4257 - val_accuracy: 0.5396
Epoch 8/50
119997/119997 [==============================] - 4s 35us/sample - loss: 0.5543 - accuracy: 0.6653 - val_loss: 2.5890 - val_accuracy: 0.5586
Epoch 9/50
119997/119997 [==============================] - 4s 35us/sample - loss: 0.5533 - accuracy: 0.6663 - val_loss: 2.7997 - val_accuracy: 0.5332
Epoch 10/50
119997/119997 [==============================] - 4s 35us/sample - loss: 0.5499 - accuracy: 0.6698 - val_loss: 3.6632 - val_accuracy: 0.5543
Epoch 11/50
119997/119997 [==============================] - 4s 35us/sample - loss: 0.6091 - accuracy: 0.6170 - val_loss: 3.4861 - val_accuracy: 0.5350
Epoch 12/50
119997/119997 [==============================] - 4s 36us/sample - loss: 0.6022 - accuracy: 0.6232 - val_loss: 5.0744 - val_accuracy: 0.5459
Epoch 13/50
119997/119997 [==============================] - 4s 35us/sample - loss: 0.5937 - accuracy:

Epoch 9/50
119997/119997 [==============================] - 4s 36us/sample - loss: 0.5803 - accuracy: 0.6435 - val_loss: 1.5980 - val_accuracy: 0.4985
Epoch 10/50
119997/119997 [==============================] - 4s 35us/sample - loss: 0.6005 - accuracy: 0.6215 - val_loss: 1.5110 - val_accuracy: 0.4993
Epoch 11/50
119997/119997 [==============================] - 4s 37us/sample - loss: 0.5815 - accuracy: 0.6421 - val_loss: 1.8409 - val_accuracy: 0.5053
Epoch 12/50
119997/119997 [==============================] - 4s 36us/sample - loss: 0.5668 - accuracy: 0.6566 - val_loss: 1.6294 - val_accuracy: 0.5052
Epoch 13/50
119997/119997 [==============================] - 4s 36us/sample - loss: 0.5680 - accuracy: 0.6561 - val_loss: 1.6803 - val_accuracy: 0.5043
Epoch 14/50
119997/119997 [==============================] - 4s 35us/sample - loss: 0.5556 - accuracy: 0.6666 - val_loss: 1.5418 - val_accuracy: 0.5186
Epoch 15/50
119997/119997 [==============================] - 4s 35us/sample - loss: 0.557

...

Epoch 16/50
119997/119997 [==============================] - 5s 44us/sample - loss: 0.5495 - accuracy: 0.6716 - val_loss: 1.4729 - val_accuracy: 0.6330
Epoch 17/50
119997/119997 [==============================] - 6s 47us/sample - loss: 0.5487 - accuracy: 0.6715 - val_loss: 1.2620 - val_accuracy: 0.5983
Epoch 18/50
119997/119997 [==============================] - 5s 44us/sample - loss: 0.5484 - accuracy: 0.6712 - val_loss: 1.6114 - val_accuracy: 0.6270
Epoch 19/50
119997/119997 [==============================] - 5s 44us/sample - loss: 0.5465 - accuracy: 0.6718 - val_loss: 1.3967 - val_accuracy: 0.6327
Epoch 20/50
119997/119997 [==============================] - 6s 49us/sample - loss: 0.5464 - accuracy: 0.6722 - val_loss: 1.2839 - val_accuracy: 0.6446
Epoch 21/50
119997/119997 [==============================] - 5s 45us/sample - loss: 0.5426 - accuracy: 0.6789 - val_loss: 1.3084 - val_accuracy: 0.6138
Epoch 22/50
119997/119997 [==============================] - 6s 49us/sample - loss: 0.53

Epoch 19/50
119997/119997 [==============================] - 5s 42us/sample - loss: 0.6424 - accuracy: 0.5665 - val_loss: 2.3047 - val_accuracy: 0.5576
Epoch 20/50
119997/119997 [==============================] - 5s 41us/sample - loss: 0.6366 - accuracy: 0.5687 - val_loss: 1.7129 - val_accuracy: 0.5614
Epoch 21/50
119997/119997 [==============================] - 5s 39us/sample - loss: 0.6336 - accuracy: 0.5768 - val_loss: 2.3684 - val_accuracy: 0.5581
Epoch 22/50
119997/119997 [==============================] - 6s 46us/sample - loss: 0.6323 - accuracy: 0.5789 - val_loss: 1.9169 - val_accuracy: 0.5596
Epoch 23/50
119997/119997 [==============================] - 5s 39us/sample - loss: 0.6339 - accuracy: 0.5730 - val_loss: 2.0865 - val_accuracy: 0.5454
Epoch 24/50
119997/119997 [==============================] - 5s 38us/sample - loss: 0.6320 - accuracy: 0.5870 - val_loss: 2.3938 - val_accuracy: 0.5588
Epoch 25/50
119997/119997 [==============================] - 5s 38us/sample - loss: 0.63

119997/119997 [==============================] - 6s 54us/sample - loss: 0.6893 - accuracy: 0.5049 - val_loss: 0.6716 - val_accuracy: 0.5339
Epoch 22/50
119997/119997 [==============================] - 6s 50us/sample - loss: 0.6892 - accuracy: 0.5045 - val_loss: 0.6707 - val_accuracy: 0.5563
Epoch 23/50
119997/119997 [==============================] - 7s 54us/sample - loss: 0.6894 - accuracy: 0.5060 - val_loss: 0.6672 - val_accuracy: 0.5414
Epoch 24/50
119997/119997 [==============================] - 6s 48us/sample - loss: 0.6893 - accuracy: 0.5050 - val_loss: 0.6687 - val_accuracy: 0.5373
Epoch 25/50
119997/119997 [==============================] - 7s 57us/sample - loss: 0.6918 - accuracy: 0.5046 - val_loss: 0.6743 - val_accuracy: 0.5287
Epoch 26/50
119997/119997 [==============================] - 6s 53us/sample - loss: 0.6892 - accuracy: 0.5055 - val_loss: 0.6700 - val_accuracy: 0.5380
Epoch 27/50
119997/119997 [==============================] - 6s 51us/sample - loss: 0.6892 - accurac

119997/119997 [==============================] - 5s 45us/sample - loss: 0.5075 - accuracy: 0.7156 - val_loss: 16.5546 - val_accuracy: 0.5137
Epoch 24/50
119997/119997 [==============================] - 6s 49us/sample - loss: 0.5085 - accuracy: 0.7134 - val_loss: 9.7050 - val_accuracy: 0.5184
Epoch 25/50
119997/119997 [==============================] - 6s 46us/sample - loss: 0.5040 - accuracy: 0.7193 - val_loss: 15.0641 - val_accuracy: 0.5216
Epoch 26/50
119997/119997 [==============================] - 5s 45us/sample - loss: 0.5057 - accuracy: 0.7178 - val_loss: 8.2435 - val_accuracy: 0.5269
Epoch 27/50
119997/119997 [==============================] - 5s 45us/sample - loss: 0.5034 - accuracy: 0.7188 - val_loss: 17.0218 - val_accuracy: 0.5217
Epoch 28/50
119997/119997 [==============================] - 5s 45us/sample - loss: 0.5047 - accuracy: 0.7147 - val_loss: 18.4233 - val_accuracy: 0.5117
Epoch 29/50
119997/119997 [==============================] - 6s 49us/sample - loss: 0.5093 - acc

119997/119997 [==============================] - 6s 52us/sample - loss: 0.6893 - accuracy: 0.5047 - val_loss: 0.6658 - val_accuracy: 0.5459
Epoch 26/50
119997/119997 [==============================] - 7s 55us/sample - loss: 0.6892 - accuracy: 0.5045 - val_loss: 0.6721 - val_accuracy: 0.5315
Epoch 27/50
119997/119997 [==============================] - 6s 52us/sample - loss: 0.6892 - accuracy: 0.5051 - val_loss: 0.6792 - val_accuracy: 0.5387
Epoch 28/50
119997/119997 [==============================] - 6s 53us/sample - loss: 0.6894 - accuracy: 0.5057 - val_loss: 0.6748 - val_accuracy: 0.5306
Epoch 29/50
119997/119997 [==============================] - 6s 52us/sample - loss: 0.6892 - accuracy: 0.5054 - val_loss: 0.6787 - val_accuracy: 0.4987
Epoch 30/50
119997/119997 [==============================] - 6s 51us/sample - loss: 0.6893 - accuracy: 0.5054 - val_loss: 0.6739 - val_accuracy: 0.5295
Epoch 31/50
119997/119997 [==============================] - 7s 56us/sample - loss: 0.6890 - accurac

In [11]:
for i in range(2):
    print(i)

0
1


In [10]:
for i in range(1):print(i)

0


In [ ]:
for i